<a href="https://colab.research.google.com/github/seharkansal/ML-algorithms-from-Scratch-/blob/main/decision_tree_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

# one node class and one decision tree class
import numpy as np
from collections import Counter
class Node:
  #This is the constructor method that sets up each node with optional attributes
  def __init__(self,feature=None,threshold=None,left=None,right=None,*,value=None):
    self.feature=feature #The question to ask
    self.threshold=threshold #The value to split on
    self.left=left #The node representing the "yes" branch of the question
    self.right=right #The node representing the "no" branch of the question
    self.value=value #The final decision (e.g., "car") if this is a leaf node

  def is_leaf_node(self):
    #This method checks if the node is a leaf node (i.e., it contains a final decision rather than a question).
    return self.value is not None

class DecisionTree:
  def __init__(self,min_samples_split=2,max_depth=100,n_features=None):
    self.min_samples_split=min_samples_split #The minimum number of samples needed to split a node
    self.max_depth=max_depth #The maximum depth of the tree (how many questions it can ask)
    self.n_features=n_features #The number of features to consider when splitting
    self.root=None

  def fit(self,X,y):
    self.n_features=X.shape[1] if not self.n_features else min(X.shape[1],self.n_features) #Sets the number of features to use. If n_features is not specified, it uses all features
    self.root=self._grow_tree(X,y)

  def _grow_tree(self,X,y,depth=0):
    n_samples,n_feats=X.shape
    n_labels=len(np.unique(y))
    if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
      '''
      Checks if we should stop growing the tree. Stops if:
      The tree has reached its maximum depth.
      All samples have the same label.
      There are too few samples to split.
      '''
      leaf_value=self._most_common_label(y) #If stopping, create a leaf node with the most common label.
      return Node(value=leaf_value)

    feat_idxs=np.random.choice(n_feats,self.n_features,replace=False)

      # Finds the best feature and threshold to split the data.
    best_feature, best_thresh=self._best_split(X,y,feat_idxs)

       #create child nodes
    left_idxs,right_idxs=self._split(X[:, best_feature], best_thresh)
    left=self._grow_tree(X[left_idxs, :],y[left_idxs],depth+1)
    right=self._grow_tree(X[right_idxs, :],y[right_idxs],depth+1)
    return Node(best_feature,best_thresh,left,right)

  def _best_split(self,X,y,feat_idxs):
    #Initializes the best gain as a very low number.
    best_gain=-1
    #Initializes the best feature index and threshold.
    split_idx,split_threshold=None,None
    for feat_idx in feat_idxs:
      X_column=X[:, feat_idx] #Gets the column of data for the current feature.
      thresholds=np.unique(X_column) #Gets all unique values in the feature column

      for thr in thresholds:
        gain=self.information_gain(X,y,X_column,thr) #Calculates how good this split is
        if gain>best_gain:
          best_gain=gain
          split_idx=feat_idx
          split_threshold=thr
      return split_idx, split_threshold

  # Calculates how much better the data is after a split.
  def information_gain(self,X,y,X_column,threshold):
    parent_entropy=self.entropy(y) #Calculates how mixed up the labels are before the split
    left_idxs,right_idxs=self._split(X_column,threshold) #Splits the data into left and right parts based on the threshold

    if len(left_idxs)==0 or len(right_idxs)==0:
      #If any split is empty, return 0 (not a good split).
      return 0

    n=len(y)
    n_l,n_r=len(left_idxs),len(right_idxs) #Number of samples in each split
    e_l,e_r=self.entropy(y[left_idxs]),self.entropy(y[right_idxs])
    child_entropy=((n_l)/n)*e_l + (n_r/n)*e_r #Weighted average of the entropies of the splits
    information_gain=parent_entropy-child_entropy
    return information_gain

  def _split(self,X_column,split_thresh):
    left_idxs=np.argwhere(X_column<=split_thresh).flatten()
    '''
    Finds the indices where the values in X_column are less or greater than split_thresh.
    These indices correspond to the left or right branch of the split.
    '''
    right_idxs=np.argwhere(X_column>=split_thresh).flatten()
    return left_idxs, right_idxs


    '''
    Counts the number of occurrences of each label in y.
    hist is an array where each element is the count of a specific label.
    Calculates the probability of each label by dividing the count by the total number of labels.
    Calculates the entropy using the formula.
    Entropy measures the uncertainty or randomness.
    The sum is computed over all non-zero probabilities.
    '''
  def entropy(self, y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log(p) for p in ps if p>0])

  def _most_common_label(self,y):
    counter=Counter(y)
    value=counter.most_common(1)[0][0]
    '''
    Finds the most common label.
    most_common(1) returns a list of the most common labels with their counts,
    and [0][0] extracts just the label
    '''
    return value

  def predict(self,X):
    return np.array([self._traverse_tree(x, self.root) for x in X])
    '''
      For each data point x in X, it traverses the decision tree from the root node and makes a prediction.
      It collects all predictions into a NumPy array.
    '''


    #This function navigates through the tree to make a prediction
  def _traverse_tree(self,x,node):
      #Checks if the current node is a leaf node (i.e., it has a final decision)
      if node.is_leaf_node():
        return node.value
        #If it’s a leaf node, return the final decision (value) at this node

      #Checks if the feature value of x is less than or equal to the threshold.
      #This decides whether to go left or right in the tree
      if x[node.feature]<=node.threshold:
        return self._traverse_tree(x,node.left)
      return self._traverse_tree(x,node.right)




In [6]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np


data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)

clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

acc = accuracy(y_test, predictions)
print(acc)

0.8859649122807017
